In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import json
import re
import nltk
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import preprocess_string
import pandas as pd
import time
import pdb

from gensim import corpora
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
from gensim.models import LdaModel
import pyLDAvis.gensim               #pyLDAvis.gensim_models
from scipy.spatial import distance

#nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
train = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv')
test = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv')
title = pd.read_csv('/kaggle/input/cpc-codes/titles.csv')

In [ ]:
print(len(train),len(test))

# Pre-processing

In [ ]:
frames = [train, test]
all = pd.concat(frames, keys=['train', 'test'])
all

In [ ]:
df_test = all.loc['test']
df_test

In [ ]:
df_LEFT_JOIN = pd.merge(all, title, left_on='context', right_on='code', how='left')
df_LEFT_JOIN

In [ ]:
data = df_LEFT_JOIN.drop(['code','section','class','subclass','group','main_group'], axis=1)
data = data[['id','anchor','target','context','title','score']]
data

In [ ]:
def clean_text(d):
    d = str(d)
    pattern = r'[^a-zA-Z\'\s]'
    text = re.sub(pattern, '', d)
    text = text.lower()
    #text = [word for word in text if len(word)>2]
    return ' '.join([w for w in text.split() if len(w) > 3])# 3이하인거 버리기

def preprocessing(d):
    return preprocess_string(d)

def data_load(data):
    df = pd.DataFrame({'text': data['total']})
    tokenized_text = df['text'].apply(preprocessing)
    tokenized_text = tokenized_text.to_list()
    return tokenized_text 

def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(doc):
            if j == 0:  
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
            else:
                break
    return(topic_table)

def create_output(topictable, topicnum):
    total_topic_list=list()
    
    for topic in range(len(topictable["각 토픽의 비중"])):
        topic_dict=dict()
        tmp_list = list()
        for k in range(len(topictable["각 토픽의 비중"][topic])):
            topic_dict[f'topic_{int(topictable["각 토픽의 비중"][topic][k][0])}'] = float(topictable["각 토픽의 비중"][topic][k][1])
            tmp_list.append(int(topictable["각 토픽의 비중"][topic][k][0]))

        num_list = [i for i in range(topicnum)]
        num_list = [i for i in num_list if i not in tmp_list]   ## 있는 토픽 제거

        for n in num_list:
            topic_dict[f'topic_{n}'] = 0.0      ## 확률 0인 토픽 넣어줌
        sorted_dict = sorted(topic_dict.items())
        total_topic_list.append(dict(sorted_dict))

    with open(f"./out_t9_total1.json", 'w') as topic_file:
        json.dump(total_topic_list, topic_file, indent='\t')

In [ ]:
data['title'] = data['title'].apply(clean_text)

In [ ]:
origin_data = data
origin_data

In [ ]:
total_df = pd.DataFrame(columns=['id','total'])
total_df['total'] = data['anchor'] + " " + data['target'] + " " +  data['title']
total_df['id'] = data['id']
total_df

# LDA Modeling

In [ ]:
doc_token = data_load(total_df)
LDA_new_list = doc_token

## Modeling
dictionary = corpora.Dictionary(doc_token)
corpus = [dictionary.doc2bow(text) for text in doc_token] #문서를 bag-of-words 형태로 바꾼것

NUM_TOPICS = 9
NUM_PASSES = 30
lda_model =LdaModel(corpus, num_topics = NUM_TOPICS, id2word = dictionary, passes = NUM_PASSES)
topics = lda_model.print_topics(num_words = 10)
    
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.save_html(vis, 'LDAvis_t9_total1.html')

topictable = make_topictable_per_doc(lda_model, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

create_output(topictable, NUM_TOPICS)